In [1]:
import os
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import pickle

# Function to load and preprocess images
def load_images_from_folder(folder, image_size=(64, 64)):
    images = []
    labels = []
    for label in os.listdir(folder):
        label_folder = os.path.join(folder, label)
        for filename in os.listdir(label_folder):
            img_path = os.path.join(label_folder, filename)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            if img is not None:
                img_resized = cv2.resize(img, image_size)
                images.append(img_resized.flatten())
                labels.append(label)
    return np.array(images), np.array(labels)

# Load dataset
X, y = load_images_from_folder('medical')

# Encode labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Initialize and train the SVM model
svm_model = SVC(kernel='linear', C=1.0, random_state=42)
svm_model.fit(X_train, y_train)

# Evaluate the model
y_pred = svm_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Model accuracy: {accuracy * 100:.2f}%")

# Save the model
model = {'svm_model': svm_model, 'scaler': scaler, 'label_encoder': label_encoder}
with open('svm_medical_model.pkl', 'wb') as file:
    pickle.dump(model, file)


Model accuracy: 99.52%


In [4]:
import os
import numpy as np
import pandas as pd
import cv2

# Function to load and preprocess images
def load_images_from_folder(folder, image_size=(64, 64)):
    images = []
    labels = []
    for label in os.listdir(folder):
        label_folder = os.path.join(folder, label)
        for filename in os.listdir(label_folder):
            img_path = os.path.join(label_folder, filename)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            if img is not None:
                img_resized = cv2.resize(img, image_size)
                images.append(img_resized.flatten())
                labels.append(label)
    return np.array(images), np.array(labels)

# Load dataset
X, y = load_images_from_folder('medical')

# Encode labels
label_mapping = {label: idx for idx, label in enumerate(np.unique(y))}
y_encoded = np.array([label_mapping[label] for label in y])

# Save label mapping
pd.DataFrame(list(label_mapping.items()), columns=["Label", "Index"]).to_csv("label_mapping.csv", index=False)

# Split data
def train_test_split(X, y, test_size=0.2, random_state=None):
    if random_state:
        np.random.seed(random_state)
    indices = np.random.permutation(len(X))
    test_size = int(len(X) * test_size)
    test_indices = indices[:test_size]
    train_indices = indices[test_size:]
    return X[train_indices], X[test_indices], y[train_indices], y[test_indices]

X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Feature Scaling (Mean Normalization)
X_mean = np.mean(X_train, axis=0)
X_std = np.std(X_train, axis=0)
X_train = (X_train - X_mean) / (X_std + 1e-8)
X_test = (X_test - X_mean) / (X_std + 1e-8)

# Softmax Regression Model
class SoftmaxRegression:
    def __init__(self, learning_rate=0.01, n_iters=1000):
        self.lr = learning_rate
        self.n_iters = n_iters
        self.W = None
        self.b = None

    def softmax(self, z):
        exp_z = np.exp(z - np.max(z, axis=1, keepdims=True))  # Stability trick
        return exp_z / np.sum(exp_z, axis=1, keepdims=True)

    def one_hot_encode(self, y, num_classes):
        m = y.shape[0]
        one_hot = np.zeros((m, num_classes))
        one_hot[np.arange(m), y] = 1
        return one_hot

    def fit(self, X, y):
        n_samples, n_features = X.shape
        n_classes = len(np.unique(y))
        self.W = np.zeros((n_features, n_classes))
        self.b = np.zeros((1, n_classes))

        y_one_hot = self.one_hot_encode(y, n_classes)

        for _ in range(self.n_iters):
            z = np.dot(X, self.W) + self.b
            y_pred = self.softmax(z)
            
            # Compute gradients
            dw = (1 / n_samples) * np.dot(X.T, (y_pred - y_one_hot))
            db = (1 / n_samples) * np.sum(y_pred - y_one_hot, axis=0)

            # Update weights and bias
            self.W -= self.lr * dw
            self.b -= self.lr * db

    def predict(self, X):
        z = np.dot(X, self.W) + self.b
        y_pred = self.softmax(z)
        return np.argmax(y_pred, axis=1)

# Train the model
model = SoftmaxRegression(learning_rate=0.01, n_iters=2000)
model.fit(X_train, y_train)

# Save model manually
np.savetxt("softmax_weights.csv", model.W, delimiter=",")
np.savetxt("softmax_bias.csv", model.b, delimiter=",")

# Evaluate the model
def accuracy(y_true, y_pred):
    return np.mean(y_true == y_pred)

y_pred = model.predict(X_test)
print(f"Model accuracy: {accuracy(y_test, y_pred) * 100:.2f}%")

Model accuracy: 99.28%


In [7]:
import os
import numpy as np
import pandas as pd
import cv2

# Function to load and preprocess images
def load_images_from_folder(folder, image_size=(64, 64)):
    images = []
    labels = []
    for label in os.listdir(folder):
        label_folder = os.path.join(folder, label)
        for filename in os.listdir(label_folder):
            img_path = os.path.join(label_folder, filename)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            if img is not None:
                img_resized = cv2.resize(img, image_size)
                images.append(img_resized.flatten())
                labels.append(label)
    return np.array(images), np.array(labels)

# Load dataset
X, y = load_images_from_folder('medical')

# Encode labels
label_mapping = {label: idx for idx, label in enumerate(np.unique(y))}
y_encoded = np.array([label_mapping[label] for label in y])

# Save label mapping
pd.DataFrame(list(label_mapping.items()), columns=["Label", "Index"]).to_csv("label_mapping.csv", index=False)

# Split data
def train_test_split(X, y, test_size=0.2, random_state=None):
    if random_state:
        np.random.seed(random_state)
    indices = np.random.permutation(len(X))
    test_size = int(len(X) * test_size)
    test_indices = indices[:test_size]
    train_indices = indices[test_size:]
    return X[train_indices], X[test_indices], y[train_indices], y[test_indices]

X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Feature Scaling (Mean Normalization)
X_mean = np.mean(X_train, axis=0)
X_std = np.std(X_train, axis=0)
X_train = (X_train - X_mean) / (X_std + 1e-8)
X_test = (X_test - X_mean) / (X_std + 1e-8)

# Save Mean and Std for Future Use
np.savetxt("train_mean.csv", X_mean, delimiter=",")
np.savetxt("train_std.csv", X_std, delimiter=",")

# Softmax Regression Model
class SoftmaxRegression:
    def __init__(self, learning_rate=0.01, n_iters=2000):
        self.lr = learning_rate
        self.n_iters = n_iters
        self.W = None
        self.b = None

    def softmax(self, z):
        exp_z = np.exp(z - np.max(z, axis=1, keepdims=True))  # Stability trick
        return exp_z / np.sum(exp_z, axis=1, keepdims=True)

    def one_hot_encode(self, y, num_classes):
        m = y.shape[0]
        one_hot = np.zeros((m, num_classes))
        one_hot[np.arange(m), y] = 1
        return one_hot

    def fit(self, X, y):
        n_samples, n_features = X.shape
        n_classes = len(np.unique(y))
        self.W = np.zeros((n_features, n_classes))
        self.b = np.zeros((1, n_classes))

        y_one_hot = self.one_hot_encode(y, n_classes)

        for _ in range(self.n_iters):
            z = np.dot(X, self.W) + self.b
            y_pred = self.softmax(z)
            
            # Compute gradients
            dw = (1 / n_samples) * np.dot(X.T, (y_pred - y_one_hot))
            db = (1 / n_samples) * np.sum(y_pred - y_one_hot, axis=0)

            # Update weights and bias
            self.W -= self.lr * dw
            self.b -= self.lr * db

    def predict(self, X):
        z = np.dot(X, self.W) + self.b
        y_pred = self.softmax(z)
        return np.argmax(y_pred, axis=1)

# Train the model
model = SoftmaxRegression(learning_rate=0.01, n_iters=2000)
model.fit(X_train, y_train)

# Save Model Weights
np.savetxt("softmax_weights.csv", model.W, delimiter=",")
np.savetxt("softmax_bias.csv", model.b, delimiter=",")

# Evaluate the Model
def accuracy(y_true, y_pred):
    return np.mean(y_true == y_pred)

y_pred = model.predict(X_test)
print(f"Model accuracy: {accuracy(y_test, y_pred) * 100:.2f}%")

Model accuracy: 99.28%


In [9]:
import numpy as np
import pandas as pd
import cv2
import os

# Load model parameters
W = np.loadtxt("softmax_weights.csv", delimiter=",")
b = np.loadtxt("softmax_bias.csv", delimiter=",")

# Load label mapping
label_mapping_df = pd.read_csv("label_mapping.csv")
label_mapping = dict(zip(label_mapping_df["Index"], label_mapping_df["Label"]))

# Function to preprocess images for testing
def load_and_preprocess_image(image_path, image_size=(64, 64)):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if img is not None:
        img_resized = cv2.resize(img, image_size)
        return img_resized.flatten()
    else:
        raise ValueError(f"Image at {image_path} could not be loaded.")

# Load and preprocess new images
new_images_folder = 'sample'
new_images = []
image_paths = []
for filename in os.listdir(new_images_folder):
    img_path = os.path.join(new_images_folder, filename)
    try:
        img_processed = load_and_preprocess_image(img_path)
        new_images.append(img_processed)
        image_paths.append(filename)
    except ValueError as e:
        print(e)

new_images = np.array(new_images)

# Feature Scaling
X_mean = np.loadtxt("train_mean.csv", delimiter=",")
X_std = np.loadtxt("train_std.csv", delimiter=",")
new_images = (new_images - X_mean) / (X_std + 1e-8)

# Predict function using the saved model
def softmax(z):
    exp_z = np.exp(z - np.max(z, axis=1, keepdims=True))
    return exp_z / np.sum(exp_z, axis=1, keepdims=True)

def predict(X):
    z = np.dot(X, W) + b
    y_pred = softmax(z)
    return np.argmax(y_pred, axis=1)

# Predict the classes of the new images
predictions = predict(new_images)

# Convert predictions back to original class names
predicted_labels = [label_mapping[int(label)] for label in predictions]

# Display results
for img_name, label in zip(image_paths, predicted_labels):
    print(f"Image: {img_name} --> Predicted Label: {label}")


Image: samp.png --> Predicted Label: ChestCT
Image: sample.jpeg --> Predicted Label: HeadCT
